In [86]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

#### Fetch xml sitemap

In [87]:
sitemap = pd.read_xml('https://www.rtbf.be/site-map/articles.xml')
sitemap.head()

,loc,lastmod,changefreq,news,image
0,https://www.rtbf.be/article/200-personnes-a-l-...,2023-07-31T18:06:09.000Z,hourly,NaN,NaN
1,https://www.rtbf.be/article/guerre-en-ukraine-...,2023-07-31T18:01:53.000Z,hourly,NaN,NaN
2,https://www.rtbf.be/article/direct-guerre-en-u...,2023-07-31T17:51:34.000Z,hourly,NaN,NaN
3,https://www.rtbf.be/article/stan-wawrinka-des-...,2023-07-31T17:51:16.000Z,hourly,NaN,NaN
4,https://www.rtbf.be/article/deces-a-tomorrowla...,2023-07-31T17:33:05.000Z,hourly,NaN,NaN


#### Keep only two first columns and change their name

In [88]:
df = sitemap.drop(['changefreq', 'news', 'image'],axis=1)
df.rename(columns={'loc':'source_url', 'lastmod':'date'}, inplace=True)
df.head()

,source_url,date
0,https://www.rtbf.be/article/200-personnes-a-l-...,2023-07-31T18:06:09.000Z
1,https://www.rtbf.be/article/guerre-en-ukraine-...,2023-07-31T18:01:53.000Z
2,https://www.rtbf.be/article/direct-guerre-en-u...,2023-07-31T17:51:34.000Z
3,https://www.rtbf.be/article/stan-wawrinka-des-...,2023-07-31T17:51:16.000Z
4,https://www.rtbf.be/article/deces-a-tomorrowla...,2023-07-31T17:33:05.000Z


#### Keep only date in 'date' column

In [99]:
df['date'].replace({r"T.+": ''}, inplace=True, regex=True)
df.head()

,source_url,date
0,https://www.rtbf.be/article/200-personnes-a-l-...,2023-07-31
1,https://www.rtbf.be/article/guerre-en-ukraine-...,2023-07-31
2,https://www.rtbf.be/article/direct-guerre-en-u...,2023-07-31
3,https://www.rtbf.be/article/stan-wawrinka-des-...,2023-07-31
4,https://www.rtbf.be/article/deces-a-tomorrowla...,2023-07-31


In [89]:
urls = df['source_url']
urls.head()

0    https://www.rtbf.be/article/200-personnes-a-l-...
1    https://www.rtbf.be/article/guerre-en-ukraine-...
2    https://www.rtbf.be/article/direct-guerre-en-u...
3    https://www.rtbf.be/article/stan-wawrinka-des-...
4    https://www.rtbf.be/article/deces-a-tomorrowla...
Name: source_url, dtype: object

In [90]:
article_one = urls[0]
print(article_one)

https://www.rtbf.be/article/200-personnes-a-l-ouverture-de-la-crypte-royale-pour-rendre-hommage-au-roi-baudouin-11235207


In [91]:
response = requests.get(article_one)
response.text

'<!DOCTYPE html><html lang="fr" dir="ltr"><head><meta charSet="utf-8"/><meta name="viewport" content="initial-scale=1.0, width=device-width, minimum-scale=1"/><meta property="al:android:url" content="rtbf://applinks/article/200-personnes-a-l-ouverture-de-la-crypte-royale-pour-rendre-hommage-au-roi-baudouin-11235207"/><meta property="al:android:app_name" content="RTBF"/><meta property="al:android:package" content="be.rtbf"/><meta property="al:web:url" content="https://rtbf.be"/><link rel="preconnect" href="https://ds.static.rtbf.be"/><link rel="dns-prefetch" href="https://ds.static.rtbf.be"/><link rel="preconnect" href="https://ds1.static.rtbf.be"/><link rel="dns-prefetch" href="https://ds1.static.rtbf.be"/><link rel="preconnect" href="https://static-oaos.rtbf.be"/><link rel="dns-prefetch" href="https://static-oaos.rtbf.be"/><link rel="preconnect" href="https://sgc.static.rtbf.be"/><link rel="dns-prefetch" href="https://sgc.static.rtbf.be"/><link rel="preconnect" href="https://www.stati

In [92]:
soup = bs(response.content, 'html.parser')


#### Find article title

In [93]:
title = soup.find('h1').text
print(title)

200 personnes à l’ouverture de la crypte royale pour rendre hommage au Roi Baudouin


#### Find article title function

In [94]:
def find_article_title(url:str) -> str: 
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    article_title = soup.find('h1').text
    return article_title

In [95]:
print(find_article_title(article_one))


200 personnes à l’ouverture de la crypte royale pour rendre hommage au Roi Baudouin


#### Find article text

In [96]:
paragraphs = soup.find_all('p', attrs={'class': None})
article_text = ''
for paragraph in paragraphs:
   article_text += paragraph.text
   article_text += '\n'
print(article_text)   

Ils étaient environ deux cents à attendre que les grilles de la crypte royale de Notre-Dame de Laeken s’ouvrent enfin ce lundi 31 août. Derrière elles, ce sont les rois et les reines de notre pays qui reposent dans une grande et solennelle salle ornée de marbres polychromes. Face à l’impressionnant monument érigé pour Léopold 1er et la Reine Louise-Marie, un tombeau plus modeste abrite les sépultures de Baudouin et Fabiola. Devant elle, une élégante gerbe de roses blanches relevées des couleurs du drapeau belge a été disposée.
À 14 heures, les premiers visiteurs sont enfin autorisés à entrer. Devant le tombeau, les discussions s’arrêtent, certains osent sortir leurs téléphones pour prendre une photo, d’autres regardent avec insistance le cliché en noir et blanc des deux souverains pris peu de temps après leur mariage.
Dans l’assistance, des Wallons, des Bruxellois et des Flamands sont réunis. Un couple d’Anversois est venu en voiture pour l’occasion. Des passionnés d’histoire qui salue

#### Find article text function

In [97]:
def find_article_text(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    paragraphs = soup.find_all('p', attrs={'class': None})
    article_text = ''
    for paragraph in paragraphs:
        article_text += paragraph.text
    return article_text    

In [98]:
print(find_article_text(article_one))

Ils étaient environ deux cents à attendre que les grilles de la crypte royale de Notre-Dame de Laeken s’ouvrent enfin ce lundi 31 août. Derrière elles, ce sont les rois et les reines de notre pays qui reposent dans une grande et solennelle salle ornée de marbres polychromes. Face à l’impressionnant monument érigé pour Léopold 1er et la Reine Louise-Marie, un tombeau plus modeste abrite les sépultures de Baudouin et Fabiola. Devant elle, une élégante gerbe de roses blanches relevées des couleurs du drapeau belge a été disposée.À 14 heures, les premiers visiteurs sont enfin autorisés à entrer. Devant le tombeau, les discussions s’arrêtent, certains osent sortir leurs téléphones pour prendre une photo, d’autres regardent avec insistance le cliché en noir et blanc des deux souverains pris peu de temps après leur mariage.Dans l’assistance, des Wallons, des Bruxellois et des Flamands sont réunis. Un couple d’Anversois est venu en voiture pour l’occasion. Des passionnés d’histoire qui saluent